# Method of manufactured solutions for linear elasticity

In order to test the implementation of linear elasticity on a structural level, we use the method of manufactured solutions to obtain an analytical solution for the structural problem. This is achieved by assuming an analytical expression for the displacements which is then used to calculate the strains, stresses. 

The stresses are then used to calculate the forces acting on the structure which are used to determine boundary conditions for an FEM problem. The FEM problem is then solved and the results are compared to the analytical solution.

The analytical expression for the displacement should be chosen such that it cannot be easily expressed by the FEM solution because otherwise the error might become zero and a convergence analysis becomes meaningless. 

Given a displacement field $u(x,y,z)$, the strain tensor is given by 
$$
\varepsilon = \frac{1}{2} \left( \nabla u + (\nabla u)^\top \right)
$$
and the stress tensor is given by
$$
\sigma = \lambda \mathrm{tr}(\varepsilon) I + 2 \mu \varepsilon
$$
where $\lambda$ and $\mu$ are the Lamé parameters and $I$ is the identity tensor.

Given an arbitrary displacement field $u(x,y,z)$, we can write these equations as `sympy` expressions:

In [4]:
from sympy import Function, Matrix, Max, symbols, sqrt, eye, Rational


def strain(u, x, y, z):
    jacobian = u.jacobian([x, y, z])
    return (jacobian + jacobian.T) / 2


def to_mandel_notation(strain):
    factor = sqrt(2)
    return Matrix(
        [
            strain[0, 0],
            strain[1, 1],
            strain[2, 2],
            factor * strain[0, 1],
            factor * strain[0, 2],
            factor * strain[1, 2],
        ]
    )


u, x, y, z = symbols("u x y z")
mu, lam = symbols(r"\mu \lambda")


def elasticity_matrix(mu, lam):
    return Matrix(
        [
            [lam + 2 * mu, lam, lam, 0, 0, 0],
            [lam, lam + 2 * mu, lam, 0, 0, 0],
            [lam, lam, lam + 2 * mu, 0, 0, 0],
            [0, 0, 0, 2 * mu, 0, 0],
            [0, 0, 0, 0, 2 * mu, 0],
            [0, 0, 0, 0, 0, 2 * mu],
        ]
    )


u_x = Function("u_x")(x, y, z)
u_y = Function("u_y")(x, y, z)
u_z = Function("u_z")(x, y, z)

u = Matrix([u_x, u_y, u_z])

eps = strain(u, x, y, z)
sigma = elasticity_matrix(mu, lam) * to_mandel_notation(eps)

sigma

Matrix([
[\lambda*Derivative(u_y(x, y, z), y) + \lambda*Derivative(u_z(x, y, z), z) + (\lambda + 2*\mu)*Derivative(u_x(x, y, z), x)],
[\lambda*Derivative(u_x(x, y, z), x) + \lambda*Derivative(u_z(x, y, z), z) + (\lambda + 2*\mu)*Derivative(u_y(x, y, z), y)],
[\lambda*Derivative(u_x(x, y, z), x) + \lambda*Derivative(u_y(x, y, z), y) + (\lambda + 2*\mu)*Derivative(u_z(x, y, z), z)],
[                                            2*sqrt(2)*\mu*(Derivative(u_x(x, y, z), y)/2 + Derivative(u_y(x, y, z), x)/2)],
[                                            2*sqrt(2)*\mu*(Derivative(u_x(x, y, z), z)/2 + Derivative(u_z(x, y, z), x)/2)],
[                                            2*sqrt(2)*\mu*(Derivative(u_y(x, y, z), z)/2 + Derivative(u_z(x, y, z), y)/2)]])

Now we can chose a highly nonlinear displacement field like 
$$
u(x,y,z) = [\sin(x),  \sin(x)\sin(y), \sin(x)  \sin(y) \sin(z)]
$$
and calculate the corresponding strains and stresses.

In [6]:
from sympy import sin
u_nonlinear = Matrix([sin(x), sin(x)*sin(y), sin(x)*sin(y)*sin(z)])

eps_nonlinear = strain(u_nonlinear, x, y, z)
sigma_nonlinear = elasticity_matrix(mu, lam) * to_mandel_notation(eps_nonlinear)
sigma_nonlinear

Matrix([
[\lambda*sin(x)*sin(y)*cos(z) + \lambda*sin(x)*cos(y) + (\lambda + 2*\mu)*cos(x)],
[\lambda*sin(x)*sin(y)*cos(z) + \lambda*cos(x) + (\lambda + 2*\mu)*sin(x)*cos(y)],
[\lambda*sin(x)*cos(y) + \lambda*cos(x) + (\lambda + 2*\mu)*sin(x)*sin(y)*cos(z)],
[                                                      sqrt(2)*\mu*sin(y)*cos(x)],
[                                               sqrt(2)*\mu*sin(y)*sin(z)*cos(x)],
[                                               sqrt(2)*\mu*sin(x)*sin(z)*cos(y)]])

The underlying PDE is the momentum of linear balance equation
$$
\nabla \cdot \sigma = f
$$
where $f$ is the body force.
Since we have an anylytical and differentiable solution, we can calculate the body force $f$ by inserting the analytical solution into the momentum balance equation.

In [8]:
from sympy import diff

def div(sigma):
    return Matrix(
        [
            diff(sigma[0], x) + diff(sigma[3], y) + diff(sigma[4], z),
            diff(sigma[3], x) + diff(sigma[1], y) + diff(sigma[5], z),
            diff(sigma[4], x) + diff(sigma[5], y) + diff(sigma[2], z),
        ]
    )
div(sigma_nonlinear)

Matrix([
[\lambda*sin(y)*cos(x)*cos(z) + \lambda*cos(x)*cos(y) + sqrt(2)*\mu*sin(y)*cos(x)*cos(z) + sqrt(2)*\mu*cos(x)*cos(y) - (\lambda + 2*\mu)*sin(x)],
[                 \lambda*sin(x)*cos(y)*cos(z) - sqrt(2)*\mu*sin(x)*sin(y) + sqrt(2)*\mu*sin(x)*cos(y)*cos(z) - (\lambda + 2*\mu)*sin(x)*sin(y)],
[                                                                  -2*sqrt(2)*\mu*sin(x)*sin(y)*sin(z) - (\lambda + 2*\mu)*sin(x)*sin(y)*sin(z)]])

In [14]:

str(div(sigma_nonlinear))

from sympy import parse_expr, srepr

parse_expr(srepr(div(sigma_nonlinear)), evaluate=False)

Matrix([
[\lambda*sin(y)*cos(x)*cos(z) + \lambda*cos(x)*cos(y) + sqrt(2)*\mu*sin(y)*cos(x)*cos(z) + sqrt(2)*\mu*cos(x)*cos(y) - (\lambda + 2*\mu)*sin(x)],
[                 \lambda*sin(x)*cos(y)*cos(z) - sqrt(2)*\mu*sin(x)*sin(y) + sqrt(2)*\mu*sin(x)*cos(y)*cos(z) - (\lambda + 2*\mu)*sin(x)*sin(y)],
[                                                                  -2*sqrt(2)*\mu*sin(x)*sin(y)*sin(z) - (\lambda + 2*\mu)*sin(x)*sin(y)*sin(z)]])